In [32]:
import requests
import pandas as pd

API_KEY = "13ed82c146522c3358d0f18881b5e4cf"
HEADERS = {"x-key": API_KEY, "User-Agent": "gie-client/1.0"}

# 1) Directory: list companies + facilities in Austria (AT)
r = requests.get("https://agsi.gie.eu/api", headers=HEADERS, params={"country": "AT"})
r.raise_for_status()
directory = r.json()
# Flatten company/facility list to pick an EIC
rows = []
for company in directory:
    for fac in company.get("facilities", []):
        rows.append({
            "company_name": company["name"],
            "company_eic": company["eic"],
            "facility_name": fac["name"],
            "facility_eic": fac["eic"],
            "country": fac["country"]
        })
df_dir = pd.DataFrame(rows)
print(df_dir.head())

# 2) Data: country-level AGSI for Austria
r2 = requests.get("https://agsi.gie.eu/api/data/at", headers=HEADERS, params={"from":"2025-01-01"})
r2.raise_for_status()
df_at = pd.json_normalize(r2.json()["data"])
df_at["gasDayStart"] = pd.to_datetime(df_at["gasDayStart"], utc=True)
print(df_at.tail())

# 3) Data: specific facility by EIC within a country
fac_eic = df_dir.loc[0, "facility_eic"]
r3 = requests.get(f"https://agsi.gie.eu/api/data/{fac_eic}/at", headers=HEADERS, params={"from":"2025-01-01"})
r3.raise_for_status()
df_fac = pd.json_normalize(r3.json()["data"])
df_fac["gasDayStart"] = pd.to_datetime(df_fac["gasDayStart"], utc=True)
print(df_fac.tail())


SSLError: HTTPSConnectionPool(host='agsi.gie.eu', port=443): Max retries exceeded with url: /api?country=AT (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1010)')))

In [43]:
import requests, pandas as pd, certifi, os
#import truststore
#truststore.inject_into_ssl() 

def gie(api_key, verify=True, **params):
    r = requests.get(
        "https://agsi.gie.eu/api",
        headers={"x-key": api_key, "User-Agent": "gie-client/1.0"},
        params=params, timeout=60, verify=verify
    )
    r.raise_for_status()
    j = r.json()
    df = pd.json_normalize(j.get("data", []))
    if "gasDayStart" in df.columns:
        df["gasDayStart"] = pd.to_datetime(df["gasDayStart"], utc=True)
    return j, df

API_KEY = "13ed82c146522c3358d0f18881b5e4cf"

# EU aggregate, single day  (use `date`, not `gasDayStart`)
meta_eu, df_eu = gie(API_KEY, verify=VERIFY, type="eu")

# Germany, single day
meta_de, df_de = gie(API_KEY, verify=VERIFY, country="de", date="2022-03-31")

print(df_eu.head())
print(df_de.head())


SSLError: HTTPSConnectionPool(host='agsi.gie.eu', port=443): Max retries exceeded with url: /api?type=eu (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1010)')))